In [2]:
import numpy as np
import pandas as pd
import librosa
import librosa.display
from IPython.display import Audio, display
import numpy as np

import os
import time
import cmath

import keras
import keras.layers as layers
from keras import backend as K
from keras.utils import plot_model
from keras import regularizers
from keras import layers

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [3]:
# If on google colab, run this
from google.colab import drive
drive.mount('/content/drive')

os.chdir('drive/My Drive/Projects/Audio Separation/Data')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Rewriten to use a single stream of data and on the fly creation of batch data

TODO
* Add saving best models

* Change name from frames to samples everywhere

* Go back to cqt??

* Add drop out

* Use callbacks for LR reduction and to save best models with error info

* Automatic logging of graphs, errors, models etc. for comparison
* Add graph of model
    from keras.utils import plot_model
    plot_model(model, to_file='model.png')
* Create log output function, then capture to text file (also export graphs?)

* Look for sources of noise that are close to what I want?  Or generate some clips?

* Debugging tools for comparing, listening to, and viewing clips
* Add audio quality comparisons between clean clips and cleaned clips for evaluation

* Add history error plotting to compare different learning models, topologies, etc.

* Look into streaming frames to & from file and having way more samples

* Why does https://towardsdatascience.com/audio-ai-isolating-vocals-from-stereo-music-using-convolutional-neural-networks-210532383785 only use real component?
* Evaluate different representations of complex numbers in terms of learnability

* Consider generating custom data for goal - male speakers, low voice, specific sounds

In [0]:
# Constants and settings
WINDOW_SIZE = 35  # Has to be odd
TARGET_COL = WINDOW_SIZE//2

DTYPE = 'float32'

# cqt related
FFT_BINS = 768 # function of items below
HOP_LENGTH = 512

# stft values
N_FFT = 1024 # 512 recommended for speech, music typically 2048

# cqt values
BINS_PER_OCTAVE = 12 * 8
FMIN = librosa.note_to_hz('C1')
OCTAVES = 8

In [0]:
# Idea here is that we operate on magnitude, and will just use phase from the original noisy sample

def combine_target(t):
    return (t[0:t.shape[0]//2] + 1j * t[t.shape[0]//2:]).reshape(1,(t.shape[0]//2))

def rebuild_fft(output, original_fft):
    vphase = np.vectorize(cmath.phase)
    o_phase = vphase(original_fft)
    mag = output.T
    vrect = np.vectorize(cmath.rect)
    return vrect(mag, o_phase)
    
# build up as (bins, samples) then transpose to model view of (samples, bins)
def targets_to_fft(targets):
    fft = np.empty((targets.shape[0],targets.shape[1]//2), dtype='complex64')
    for i in range(0, targets.shape[0]):
        fft[i] = combine_target(targets[i])
    return fft.T   # transpose

def get_ft(wav):
    c = librosa.cqt(wav, hop_length=HOP_LENGTH, fmin=FMIN, n_bins=OCTAVES*BINS_PER_OCTAVE, bins_per_octave=BINS_PER_OCTAVE)
    #c = librosa.stft(wav, hop_length=HOP_LENGTH, n_fft=N_FFT)
    return c

def inv_ft(ft):
    return librosa.icqt(ft, hop_length=HOP_LENGTH, bins_per_octave=BINS_PER_OCTAVE)
    #return librosa.istft(ft, hop_length=HOP_LENGTH)
    

In [0]:
# For local data
samples=100000
frames = np.memmap("f://audiodata//fsamples-"+str(samples), mode='r', dtype=DTYPE, shape=(samples, FFT_BINS, 1))
targets = np.memmap("f://audiodata//ftargets-"+str(samples), mode='r', dtype=DTYPE, shape=(samples, FFT_BINS))

In [0]:
# For collab data
samples = 300000
frames = np.memmap("fsamples-"+str(samples), mode='r', dtype=DTYPE, shape=(samples, FFT_BINS, 1))
targets = np.memmap("ftargets-"+str(samples), mode='r', dtype=DTYPE, shape=(samples, FFT_BINS))

In [0]:
# Code to generate input, target, and verification data
# From https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, input_array, target_array, batch_size=32, shuffle=True):
        'Initialization'
        self.input_array = input_array
        self.target_array = target_array
        self.dim = (WINDOW_SIZE,FFT_BINS) # Input data shape
        self.batch_size = batch_size
        self.list_IDs = list_IDs  # the set of allowed IDs to use as addresses for frames and target samples
        self.n_channels = 1
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        y = np.empty((self.batch_size, FFT_BINS), dtype=DTYPE)

        # Generate data
        half_win = WINDOW_SIZE // 2
        for i, ID in enumerate(list_IDs_temp):
            # Store sample & target
            X[i,] = self.input_array[ID-half_win:ID+half_win+1,:,:]
            y[i] = self.target_array[ID,:]

        return X, y

In [0]:
# Prepare the generator
validation_split = 0.05
batch_size = 64

# Available IDs
all_IDs = np.arange(WINDOW_SIZE//2, frames.shape[0]-WINDOW_SIZE//2)
np.random.shuffle(all_IDs)

validation_cut = int(np.floor(len(all_IDs) * (1-validation_split)))
validation_IDs = all_IDs[validation_cut:]
train_IDs = all_IDs[0:validation_cut-1]

training_generator = DataGenerator(train_IDs, frames, targets, batch_size=batch_size)
validation_generator = DataGenerator(validation_IDs, frames, targets, batch_size=batch_size)


In [9]:
# Real model (eventually)
ALPHA = 0.05
L2Reg = 0.00000 # bumping this up to 0.000005 didn't seem very good

inputs = layers.Input(shape=(WINDOW_SIZE,FFT_BINS,1)) # Full window

NARROW_VIEW = 15

# WARNING - if I put a constant in the lambda function, it causes an error when loading the model
def sub_samples(samples, n, window_size):
    half_cut = (window_size - n) // 2
    return samples[:,half_cut:-half_cut,:,:]
    
# process inputs into subsets of interest
input_slice = layers.Lambda(sub_samples, arguments = {'n':1, 'window_size':WINDOW_SIZE}, name="targetslice")(inputs)
narrow_view = layers.Lambda(sub_samples, arguments = {'n':NARROW_VIEW, 'window_size':WINDOW_SIZE}, name="narrow_view")(inputs)

# zoomed out view
broad_pool1 = layers.AveragePooling2D(pool_size=(3,5), strides=(2,3), name="broad_pool1")(inputs)
broad_conv1 = layers.Conv2D(64, kernel_size=(5,BINS_PER_OCTAVE//5), strides=(2,BINS_PER_OCTAVE//20), activity_regularizer=regularizers.l2(L2Reg), name="broad_conv1")(broad_pool1)
bc1a = layers.LeakyReLU(alpha=ALPHA)(broad_conv1)
broad_conv2 = layers.Conv2D(32, kernel_size=(3,4), strides=(1,4), activity_regularizer=regularizers.l2(L2Reg), name="broad_conv2")(bc1a)
bc2a = layers.LeakyReLU(alpha=ALPHA)(broad_conv2)
broad_pool2 = layers.MaxPooling2D(pool_size=(1,4), strides=(1,2), name="broad_pool2")(bc2a)
bp2a = layers.LeakyReLU(alpha=ALPHA)(broad_pool2)
flat_b = layers.Flatten(name="flat_b")(bp2a)

# narrow view
conv1 = layers.Conv2D(128, kernel_size=(3,BINS_PER_OCTAVE), strides=(1,BINS_PER_OCTAVE//4), activity_regularizer=regularizers.l2(L2Reg), name="conv1")(narrow_view)
c1a = layers.LeakyReLU(alpha=ALPHA)(conv1)
conv2 = layers.Conv2D(64, kernel_size=(3,4), strides=(1,2), activity_regularizer=regularizers.l2(L2Reg), name="conv2")(c1a)
c2a = layers.LeakyReLU(alpha=ALPHA)(conv2)
pool1 = layers.MaxPooling2D(pool_size=(1,8), strides=(1,4), name="pool1")(c2a)
p1a = layers.LeakyReLU(alpha=ALPHA)(pool1)
flat_n = layers.Flatten(name="flat_n")(p1a)

# Vertical features - across samples then bins
#conv3 = layers.Conv2D(64, kernel_size=(3,30), activity_regularizer=regularizers.l2(L2Reg), name="conv3")(inputs) 
#c3a = layers.LeakyReLU(alpha=ALPHA)(conv3)
#flat_v = layers.Flatten(name="flat_v")(c3a)

flat = layers.concatenate([flat_b, flat_n], name="merge_modes")

flat_in = layers.Flatten(name="flatten")(input_slice)

dense0 = layers.Dense(1000, activity_regularizer=regularizers.l2(L2Reg))(flat)
d0a = layers.LeakyReLU(alpha=ALPHA)(dense0)
merge1 = layers.concatenate([d0a, flat_in], name="merge_input")

dense1 = layers.Dense(300, activity_regularizer=regularizers.l2(L2Reg))(merge1)
d1a = keras.layers.LeakyReLU(alpha=ALPHA)(dense1)
#dense2 = layers.Dense(500, )(dense1)
#d2a = keras.layers.LeakyReLU(alpha=ALPHA)(dense2)
merged = layers.concatenate([d1a, flat_in])
outputs = layers.Dense(FFT_BINS, activation='linear')(merged)

model = keras.models.Model(inputs=[inputs], outputs=outputs)

model.compile(optimizer='Adam', loss='mse')

print(model.summary())







Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 45, 768, 1)   0                                            
__________________________________________________________________________________________________
broad_pool1 (AveragePooling2D)  (None, 22, 255, 1)   0           input_1[0][0]                    
__________________________________________________________________________________________________
narrow_view (Lambda)            (None, 15, 768, 1)   0           input_1[0][0]                    
__________________________________________________________________________________________________
broad_conv1 (Conv2D)            (None, 9, 60, 64)    6144        broad_pool1[0][0]                
______________________________________________________________________________________

In [10]:
# fit_generator
stopper = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=15, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
save_best = keras.callbacks.ModelCheckpoint("Best_model.h5", monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

hist = model.fit_generator(generator=training_generator, validation_data=validation_generator, epochs=5, callbacks = [stopper, save_best])




Epoch 1/5





4452/4452 [==============================] - 182s 41ms/step - loss: 0.0024 - val_loss: 0.0011
Epoch 2/5
4452/4452 [==============================] - 87s 19ms/step - loss: 9.0635e-04 - val_loss: 7.8675e-04
Epoch 3/5
4452/4452 [==============================] - 87s 19ms/step - loss: 7.6299e-04 - val_loss: 7.6823e-04
Epoch 4/5
4452/4452 [==============================] - 87s 19ms/step - loss: 6.8983e-04 - val_loss: 6.8618e-04
Epoch 5/5
4452/4452 [==============================] - 86s 19ms/step - loss: 6.5406e-04 - val_loss: 6.4508e-04


In [0]:
now = time.strftime('%Y-%m-%d_%H-%M')
model.save("model_"+now+".h5")

In [0]:
history = hist

# Plot training & validation accuracy values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.ylim((0, 2 * history.history['loss'][0])) # because sometimes there are stupid spikes in error
plt.show()

#plt.savefig('test.png')
print("Validation loss mean 5+ epochs {0:.4}".format(np.mean(history.history['val_loss'][5:])))
print("Lowest validation loss {0:.4}".format(np.min(history.history['val_loss'][5:])))
print("Frames ", frames.shape[0])

In [0]:
# Not required because of 'restore best weights'
#model = keras.models.load_model("Best_model.h5")

In [0]:
# some test data to hack around with
test_file = "Assets\\DataShareArchive\\Test\\Clean\\p232_010.wav"
wav, rate = librosa.core.load(test_file)
fft = get_ft(wav)
#print(fft.shape)

In [0]:
# Full round trip test
file = "p232_001.wav"

verify_file = "Assets\\DataShareArchive\\Test\\Noisy\\" + file
clean_file = "Assets\\DataShareArchive\\Test\\Clean\\" + file


# Sample output is (samples, bins) all converted to magnitude
def get_samples(file):
    wav, rate = librosa.core.load(file)
    samples = abs(get_ft(wav).T) # organized as bins, frames so we need to transpose them to frames, bins
    return samples

# Only need this for testing at this point?  Could still make it a 'top' type shared function
def clip_frames(file):
    samples = get_samples(file)
    print(samples.shape)
    frames = np.empty((samples.shape[0], WINDOW_SIZE, FFT_BINS, 1))
    half_win = WINDOW_SIZE//2
    for i in range(half_win, samples.shape[0]-half_win):
        frames[i,:,:,0] = samples[i-half_win:i+half_win+1,:]
    return frames


def clean_clip(model, n_file):
    verify_frames = clip_frames(n_file)
    output_targets = model.predict([verify_frames])
    wav, rate = librosa.core.load(n_file)
    n_fft = get_ft(wav)
    fft = rebuild_fft(output_targets, n_fft)
    return fft, inv_ft(fft)


p_fft, p_wav = clean_clip(model, verify_file)
    
wav, rate = librosa.core.load(clean_file)
c_fft = get_ft(wav)


def display_fft(ft):
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(ft), ref=np.max), y_axis='cqt_hz', x_axis='time')

print("Cleaned clip")

err_fft = c_fft - p_fft
print("Average abs err vs clean = ", np.mean(abs(err_fft)))

display_fft(p_fft)
Audio(p_wav,rate=22050)


In [0]:
def draw(wav):
    fft = get_ft(wav)
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(fft), ref=np.max), y_axis='cqt_hz', x_axis='time')
    return fft

In [0]:
wav, rate = librosa.core.load(verify_file)
n_fft = draw(wav)

err_fft = c_fft - n_fft
print("Average abs err vs clean = ", np.mean(abs(err_fft)))

print("Noisy file")
Audio(wav, rate=rate)

In [0]:
wav, rate = librosa.core.load(clean_file)
c_fft = draw(wav)

print("Clean sample")
Audio(wav, rate=rate)

In [0]:
cut_fft = n_fft-p_fft
display_fft(cut_fft)

print("Removed audio")
print("Average cut value = ", np.mean(abs(cut_fft)))

cut_wav = inv_ft(cut_fft)
Audio(cut_wav,rate=22050)


In [0]:
#%%capture cap
print(model.summary())
#with open('output.txt', 'w') as f:
#    f.write(cap.stdout)


In [0]:
fresh_wav, rate = librosa.core.load("Assets\\DataShareArchive\\Test\\Noisy\\p232_019.wav")
Audio(fresh_wav, rate=rate)

Test & reference stuff 

In [0]:
ft = clip_frames(test_file)
tt = clip_targets(test_file)

print(ft.shape)
print(tt.shape)

#Frame / target check  -  don't use first row because it might be zeroed out
r = 10
print(tt[0,r])
print(ft[0,r,TARGET_COL:TARGET_COL+1,0])

# Round trip test
new_fft = rebuild_fft(tt, fft)
print(new_fft.shape)
new_wav = inv_ft(new_fft)

librosa.display.specshow(librosa.amplitude_to_db(np.abs(new_fft), ref=np.max), y_axis='log', x_axis='time')

Audio(new_wav,rate=22050)

In [0]:
    
def show_fft(wav):
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(librosa.stft(wav)), ref=np.max), y_axis='log', x_axis='time')

def display_fft(ft):
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(fft), ref=np.max), y_axis='log', x_axis='time')

librosa.display.specshow(librosa.amplitude_to_db(np.abs(new_fft), ref=np.max), y_axis='log', x_axis='time')

print(new_fft.shape)

#show_fft(wav)
#show_fft(new_wav)
#Audio(wav, rate=22050)


In [0]:
# Test stuff for hacking around    
    
a = np.array([[1+2j, 3+4j, 1.5+2.5j],[5+6j,7+8j, 5.5+6.5j]])
print(a)
a1 = a[0]
print(a1)
ar = a1.real
ai = a1.imag

st = np.zeros((6))
st[0:3] = ar
st[3:6] = ai
print(ar.shape, ai.shape, st.shape)
print(ar, ai)
print(st)


In [0]:
# ''do nothing' model that should be able to guess outputs from outputs

inputs = layers.Input(shape=(FFT_BINS*2,))
dense1 = layers.Dense(2000, activation='relu')(inputs)
dense2 = layers.Dense(2000, activation='relu')(dense1)
#flat = layers.Flatten()(dense)
#flat_in = layers.Flatten()(inputs)
#merged = layers.concatenate([flat, flat_in])
merged = layers.concatenate([dense2, inputs])
linear = layers.Dense(2000, activation='linear')(merged)
outputs = layers.Dense(2*FFT_BINS, activation='linear')(linear)

model = keras.models.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='Adam', loss='mse')

model.fit(targets, targets, epochs=1, batch_size=32)